In [ ]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
from sklearn.cross_validation import train_test_split
import re
import operator
style.use('ggplot')

# read in csv as dataframe
# c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')
c = pd.read_csv("../blocking/candidate_set_with_added_features.csv", encoding="ISO-8859-1", index_col='_id')

c.head()

/Users/andrew/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,...,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,NaN,...,0.16,-10.0,3.0,0,0.5,1.0,0.0,1,0.0,1
75598,4363,9,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1
75600,4365,9,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\r\n The major threat is over-collectin...,['\n The Shapotou location is within a nati...,Decreasing,...,0.08,-16.0,1.0,0,0.5,1.0,0.0,1,0.0,1
78778,7543,9,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\r\n The major threat is habitat loss a...,['\n It is listed on Appendix III of the Be...,Decreasing,...,0.12,-14.0,2.0,0,0.5,1.0,0.0,0,0.0,1
83513,4363,10,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1


In [ ]:
c.columns

In [ ]:
matches = 0
nonmatches = 0;

gold = None
DS = None

while matches < 50:
    gold,unlabeled = train_test_split(c, train_size=0.1039, test_size=0.8961)

    #Automatically label the training data
    labels = []
    matches = 0;
    nonmatches = 0;
    
    for index, row in gold.iterrows():
#         print(row)

        if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
            labels.append(1)
            matches = matches + 1
        else:
            labels.append(0)
            nonmatches = nonmatches + 1

#     matches = 50
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

In [ ]:
for col in c.columns:
    print(col)

In [ ]:
# gold.to_csv('./gold.csv')
gold.to_csv('./gold.csv')
# unlabeled.to_csv('./unlabeled.csv')
unlabeled.to_csv('./unlabeled.csv')


In [ ]:
# remove unnecessary features
# write out gold and unlabeled csvs with only selected features for matcher

cols_to_keep = ['country_count_sim', 
                'status_match', 
                'country_overlap', 
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'name_name_jac_qgm_3_qgm_3',
                'country_country_jac_dlm_dc0_dlm_dc0', 
                'country_country_jac_qgm_3_qgm_3'
                ]

# FOR NOW, just use these while we wait for other features
# cols_to_keep_gold = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3',
#                 'label']
gold_reduced = gold[cols_to_keep_gold + ['label']] 

# cols_to_keep_unlabeled = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3']
unlabeled_reduced = unlabeled[cols_to_keep_unlabeled]

In [ ]:
# gold_reduced.to_csv('./gold_reduced.csv')
gold_reduced.to_csv('./gold_reduced_CME_test.csv')
# unlabeled_reduced.to_csv('./unlabeled_reduced.csv')
unlabeled_reduced.to_csv('./unlabeled_reduced.csv')

In [ ]:
#using features for matcher
train,test = train_test_split(gold, train_size=0.7, test_size=0.3)
len(train)

train_reduced = train[cols_to_keep] 
test_reduced = test[cols_to_keep]

In [ ]:
train.to_csv('./train.csv')
test.to_csv('./test.csv')
train_reduced.to_csv('./train_reduced.csv')
test_reduced.to_csv('./test_reduced.csv')